<a href="https://colab.research.google.com/github/sba22223nestorpereira/PySpark/blob/main/df_in_spark_with_gnu%20v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

#from pyspark import SparkConf, SparkContext
#conf = SparkConf().setMaster("local").setAppName("CCT App")
#sc = SparkContext(conf = conf)

from pyspark.sql import SparkSession

#spark = SparkSession.builder\
#        .master("local")\
#        .appName("Colab")\
#        .config('spark.ui.port', '4050')\
#        .getOrCreate()

spark = SparkSession.builder.master("local[*]").appName("cct").getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

spark

create a **Spark dataframe**.

In [10]:
df = spark.read.csv("train.csv", header=True, inferSchema=True)

In [11]:
df.printSchema()

root
 |-- survived: integer (nullable = true)
 |-- pclass: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: double (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- parch: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- embarked: string (nullable = true)
 |-- class: string (nullable = true)
 |-- who: string (nullable = true)
 |-- adult_male: boolean (nullable = true)
 |-- deck: string (nullable = true)
 |-- embark_town: string (nullable = true)
 |-- alive: string (nullable = true)
 |-- alone: boolean (nullable = true)



In [12]:
df.show(5)

+--------+------+------+----+-----+-----+-------+--------+-----+-----+----------+----+-----------+-----+-----+
|survived|pclass|   sex| age|sibsp|parch|   fare|embarked|class|  who|adult_male|deck|embark_town|alive|alone|
+--------+------+------+----+-----+-----+-------+--------+-----+-----+----------+----+-----------+-----+-----+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|Third|  man|      true|null|Southampton|   no|false|
|       1|     1|female|38.0|    1|    0|71.2833|       C|First|woman|     false|   C|  Cherbourg|  yes|false|
|       1|     3|female|26.0|    0|    0|  7.925|       S|Third|woman|     false|null|Southampton|  yes| true|
|       1|     1|female|35.0|    1|    0|   53.1|       S|First|woman|     false|   C|Southampton|  yes|false|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|Third|  man|      true|null|Southampton|   no| true|
+--------+------+------+----+-----+-----+-------+--------+-----+-----+----------+----+-----------+-----+-----+
o

In [13]:
df.count()

891

In [14]:
df.select("survived","pclass","age","sex").show(5)

+--------+------+----+------+
|survived|pclass| age|   sex|
+--------+------+----+------+
|       0|     3|22.0|  male|
|       1|     1|38.0|female|
|       1|     3|26.0|female|
|       1|     1|35.0|female|
|       0|     3|35.0|  male|
+--------+------+----+------+
only showing top 5 rows



In [15]:
df.describe().show()

+-------+-------------------+------------------+------+------------------+------------------+-------------------+-----------------+--------+-----+-----+----+-----------+-----+
|summary|           survived|            pclass|   sex|               age|             sibsp|              parch|             fare|embarked|class|  who|deck|embark_town|alive|
+-------+-------------------+------------------+------+------------------+------------------+-------------------+-----------------+--------+-----+-----+----+-----------+-----+
|  count|                891|               891|   891|               714|               891|                891|              891|     889|  891|  891| 203|        889|  891|
|   mean| 0.3838383838383838| 2.308641975308642|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824| 32.2042079685746|    null| null| null|null|       null| null|
| stddev|0.48659245426485753|0.8360712409770491|  null|14.526497332334035|1.1027434322934315| 0.8060572211299488|49.6934

In [16]:
df.select("sibsp").distinct().show()

+-----+
|sibsp|
+-----+
|    1|
|    3|
|    5|
|    4|
|    8|
|    2|
|    0|
+-----+



In [17]:
from pyspark.sql import functions as F
df.groupBy("pclass").agg(F.sum("survived")).show()

+------+-------------+
|pclass|sum(survived)|
+------+-------------+
|     1|          136|
|     3|          119|
|     2|           87|
+------+-------------+



In [18]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+--------+------+---+---+-----+-----+----+--------+-----+---+----------+----+-----------+-----+-----+
|survived|pclass|sex|age|sibsp|parch|fare|embarked|class|who|adult_male|deck|embark_town|alive|alone|
+--------+------+---+---+-----+-----+----+--------+-----+---+----------+----+-----------+-----+-----+
|       0|     0|  0|177|    0|    0|   0|       2|    0|  0|         0| 688|          2|    0|    0|
+--------+------+---+---+-----+-----+----+--------+-----+---+----------+----+-----------+-----+-----+



In [ ]:
df = df.fillna({'Product_Category_2':0, 'Product_Category_3':0})

In [ ]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

In [ ]:
df.write.csv("/content/drive/My Drive/AV articles/PySpark on Colab/preprocessed_data")

In [ ]:
df.rdd.getNumPartitions()

In [20]:
# Spark df to Pandas df
df_pd = df.toPandas()

# Store result
df_pd.to_csv("pandas_preprocessed_data.csv")
